# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

In [3]:
# plots
from matplotlib import pyplot as plt
import seaborn as sns
from pylab import rcParams

from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go
init_notebook_mode(connected=True)

%config InlineBackend.figure_format = 'retina'
%matplotlib inline
%pylab inline
%config InlineBackend.figure_format = 'png'
rcParams['figure.figsize'] = (16, 6)

Populating the interactive namespace from numpy and matplotlib


## Часть 1

Выберете источник данных, который вам симпатичен. По умолчанию, предлагаем использовать данные о погоде в Москве: 
https://www.gismeteo.ru/weather-moscow-4368/. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

In [4]:
import pandas as pd
import numpy as np

In [68]:
import urllib.request
from scrapy.selector import Selector


html = urllib.request.urlopen('https://www.gismeteo.ru/weather-moscow-4368')
html = html.read()
selector = Selector(text=html)

hours = selector.xpath('//div[@data-widget-id="forecast"]//div[@class="w_time"]/span/text()').extract()
hours = [int(hour) for hour in hours]

temp = selector.xpath('//div[@data-widget-id="forecast"]//div[@class="chart chart__temperature"]/div[@class="values"]/div[@class="value"]/text()').extract()
temp = [int(x.strip().replace('−', '-')) for x in temp]

wind = selector.xpath('//div[@data-widget-id="forecast"]//div[@class="w_wind"]/div/text()').extract()
wind = [list(map(int, x if len(x)>1 else x+x)) for x in list(map(lambda x: x.strip().split('-'), wind))]

prec = selector.xpath('//div[@data-widget-id="forecast"]//div[@class="widget__item w_precipitation"]/div/text()').extract()
prec = [float(x.strip().replace(',', '.')) for x in prec]

if not prec:
    prec = [0] * len(hours)

results = []

for i in range(len(hours)):
    results.append({
        'hours': hours[i],
        'temp' : temp[i],
        'prec' : prec[i],
        'wind_from' : wind[i][0],
        'wind_to' : wind[i][1]
    })

weather = pd.DataFrame(results)
    
weather

,hours,prec,temp,wind_from,wind_to
0,0,0,0,1,1
1,3,0,0,1,1
2,6,0,1,1,1
3,9,0,2,1,1
4,12,0,2,2,2
5,15,0,3,2,2
6,18,0,4,5,10
7,21,0,4,6,10


## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [79]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def morpher(s, d):
    w = morph.parse(s)[0]
    return w.make_agree_with_number(d).word

for i, row in weather.iterrows():
    print('Температура сегодня в {hours} {hours_w} составит {temp} {degree_w}. Скорость ветра {speed} {metr_w} в секунду'
          .format(hours=row['hours'],
                  hours_w=morpher('час', row['hours']),
                  temp=row['temp'],
                  degree_w=morpher('градус', row['temp']),
                  speed=row['wind_to'] if row['wind_from'] == row['wind_to'] else '%s-%s' % (row['wind_from'], row['wind_to']),
                  metr_w=morpher('метр', row['wind_to'])))

Температура сегодня в 0 часов составит 0 градусов. Скорость ветра 1 метр в секунду
Температура сегодня в 3 часа составит 0 градусов. Скорость ветра 1 метр в секунду
Температура сегодня в 6 часов составит 1 градус. Скорость ветра 1 метр в секунду
Температура сегодня в 9 часов составит 2 градуса. Скорость ветра 1 метр в секунду
Температура сегодня в 12 часов составит 2 градуса. Скорость ветра 2 метра в секунду
Температура сегодня в 15 часов составит 3 градуса. Скорость ветра 2 метра в секунду
Температура сегодня в 18 часов составит 4 градуса. Скорость ветра 5-10 метров в секунду
Температура сегодня в 21 час составит 4 градуса. Скорость ветра 6-10 метров в секунду


## Часть 3 (необязательная)
Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели (например, "сегодня похолодало на Х градусов по сравнению со вчера").